In [1]:
# load movies_model
# load posters
# load movies_test data

In [225]:
import pandas as pd
import matplotlib.pyplot as plt

from keras import models
from keras.preprocessing import image
import tensorflow as tf
from sklearn.metrics import confusion_matrix

import os

### Load Models

In [3]:
movies_model_path = 'data/movies_models/samuel_movies1533435470.765683.h5'
images_model_path = 'data/aws_models/samuel_images_model_1533409854.5854547.h5'

movies_model = models.load_model(movies_model_path)
images_model = models.load_model(images_model_path)

### Load Test Data

In [4]:
x = pd.read_csv('data/movies_x_test.csv')
y = pd.read_csv('data/movies_y_test.csv', header=None)

### Get Poster Function

In [157]:
posters_dir = 'data/posters'

posters_by_movie_id = {}
LIMIT = 10

def load_image(movie_id):
    if movie_id in posters_by_movie_id:
        return posters_by_movie_id[movie_id]

    filename = '{}.jpg'.format(movie_id)
    img_path = os.path.join(posters_dir, filename)
    
    posters_by_movie_id[movie_id] = image.load_img(img_path, target_size=(224, 224))
    return posters_by_movie_id[movie_id]

def image_to_array(img):
    img = image.img_to_array(img)
    img = img.reshape((1,) + img.shape)
    img = img / 255.
    return img

In [213]:
def show_img(id):
    plt.title(id)
    plt.imshow(load_image(id))
    plt.show()

ids_with_samuel = x.id[y[0]]
# [show_img(id) for id in ids_with_samuel]
#print(ids_with_samuel)

In [258]:
def predict_samuel_poster(movie_id):
    img = load_image(movie_id)
    img_array = image_to_array(img)
    
    
    prob = images_model.predict_proba(img_array)
    #print(prob)
    if movie_id in ids_with_samuel:
        print(prob)
    
    if prob > .999:
        return False
    elif prob < .0001:
        print('samuel')
        #plt.imshow(img)
        #plt.show()
        return True
    else:
        return None

In [259]:
def predict_samuel_movie(movie):
    prediction = movies_model.predict_classes([movie])
    return prediction[0][0] != 0

In [260]:
def predict_samuel_ensemble(movies):
    predictions_map = {
        'yes': [],
        'no': [],
    }
    predictions = []
    for index, row in movies.iterrows():
        id = int(row['id'])
        del row['id']
        
        prediction = predict_samuel_poster(id)
        #prediction = None
        
        if prediction is None:
            prediction = predict_samuel_movie([row.values])
        
        predictions.append(prediction)

        if prediction:
            predictions_map['yes'].append(id)
        elif not prediction:
            predictions_map['no'].append(id)
        else:
            print(type(prediction))
            print('unknown predict {}'.format(prediction))
    return pd.DataFrame({'predictions': predictions})

In [261]:
predictions = predict_samuel_ensemble(x)

samuel
samuel
samuel


In [266]:
# predictions
predictions[predictions.predictions]

,predictions
1,True
77,True
88,True
187,True
273,True
290,True


In [267]:
# actuals
y[y[0]]

,0
5,True
127,True
305,True
336,True
403,True
423,True
470,True


In [268]:
confusion_matrix(y.values.flatten(), predictions)

array([[468,   6],
       [  7,   0]])